In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns; sns.set()
%matplotlib inline

from imblearn.pipeline import make_pipeline, Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from category_encoders import WOEEncoder
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer, FunctionTransformer
from category_encoders import OneHotEncoder

from mlxtend.evaluate import feature_importance_permutation
from sklearn.model_selection import train_test_split

from mlxtend.feature_extraction import PrincipalComponentAnalysis
from mlxtend.preprocessing import standardize

from mlxtend.plotting import plot_pca_correlation_graph


from mlxtend.plotting import plot_decision_regions
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from sklearn.model_selection import cross_val_score

from sklearn.pipeline import make_pipeline, Pipeline

import joypy

import yfinance as yf
from finta import TA as ta
# import talib as ta

import warnings
warnings.filterwarnings("ignore")
# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


In [2]:
def get_stock(ticker):
    df = yf.Ticker(ticker)
    df = df.history(period='max')
#     df = df.history(start = '2018-10-1',end = '2019-02-10')
    for i in df.columns:
        df = df.rename(columns={str(i): str(i.lower())})
    return df
    

In [3]:
def appendData(maindf, dataarray, namesarray=None):
    if namesarray==None:
        return maindf.join(pd.DataFrame(dataarray), how='outer')
    return maindf.join(pd.DataFrame(dataarray,columns=namesarray), how='outer')

In [4]:
def create_features(df):
    '''https://byte0x20.com/python-programming-28-point-system-pt2/'''
    ### Oscillators ###
    ## RSI
    df = appendData(df,ta.RSI(df))
    ## Sto-%K
    df = appendData(df,ta.STOCH(df))
    ## CCI
    df = appendData(df,ta.CCI(df))
    ## ADX
    df = appendData(df,ta.ADX(df))
    ## DMI (Added to aid in interpreting ADX)
    df = appendData(df,ta.DMI(df, 14))
    ## Awesome
    df = appendData(df,ta.AO(df))
    ## Momentum
    df = appendData(df,ta.MOM(df,10))
    ## MACD (We rename the undescriptive "SIGNAL" here)
    df = appendData(df,ta.MACD(df)).rename(columns={"SIGNAL": "MACD SIGNAL"})
    ## Sto-RSI
    df = appendData(df,ta.STOCHRSI(df))
    ## Williams %R
    df = appendData(df,ta.WILLIAMS(df))
    ## Bull-Bear Power
    df = appendData(df,ta.EBBP(df))
    ## Ultimate (FinTA does not name this column, so we must)
    df = appendData(df,ta.UO(df),["UO"])
    ### Moving Averages ###
    sma_ema_averages = [5, 10, 20, 30, 50, 100, 200]
    ## SMA, EMA
    for i in sma_ema_averages:
      df = appendData(df,ta.SMA(df, i))
      df = appendData(df,ta.EMA(df, i))
    ## VWMA
    df = appendData(df, ta.VAMA(df, 20))
    ## Hull
    df = appendData(df,ta.HMA(df, 9))
    # Ichimoku -- Base (Kijun) and Conversion (Tenkan) Only
    df = appendData(df,ta.ICHIMOKU(df).drop(['senkou_span_a','SENKOU','CHIKOU'], axis=1))
    return df

In [5]:
def get_gauge(df):
    indicator_count = 28 # Our magic number of indicators that we will average by.
    gauge_value = 0 # Initializing our gauge value to be perfectly neutral according to our rules.
    index_value = len(df) - 1 # This will make this code mildly prettier in the way that it's currently written.
    ###### RSI
    if float(df['RSI'][index_value]) <= float(30):
        gauge_value += 1 # Adding 1 is BUY
    elif float(df['RSI'][index_value]) >= float(70):
        gauge_value -= 1 # Subtracting 1 is a SELL
    else:
        gauge_value += 0 # No value changed is a NEUTRAL
    ###### STOCH %K
    if float(df['14 period STOCH %K'][index_value]) <= float(20):
        gauge_value += 1
    elif float(df['14 period STOCH %K'][index_value]) >= float(80):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### CCI
    if float(df['20 period CCI'][index_value]) <= float(-100):
        gauge_value += 1
    elif float(df['20 period CCI'][index_value]) >= float(100):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### ADX (also using DMI)
    if (float(df['14 period ADX.'][index_value]) > float(25)) & (df['DI+'][index_value] > df['DI-'][index_value]):
        gauge_value += 1
    elif (float(df['14 period ADX.'][index_value]) > float(25)) & (df['DI+'][index_value] < df['DI-'][index_value]):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### AO
    if df['AO'][index_value] > (df['AO'][index_value - 1] + .05):
        gauge_value += 1
    elif df['AO'][index_value] < (df['AO'][index_value - 1] - .05):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### Momentum
    if df['MOM'][index_value] > (df['MOM'][index_value - 1] + .05):
        gauge_value += 1
    elif df['MOM'][index_value] < (df['MOM'][index_value - 1] - .05):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### MACD
    if (float(df['MACD'][index_value] - df['MACD SIGNAL'][index_value]) > float(0)) >= float(df['MACD'][index_value - 1] - df['MACD SIGNAL'][index_value - 1]):
        gauge_value += 1
    elif (float(df['MACD'][index_value] - df['MACD SIGNAL'][index_value]) < float(0)) <= float(df['MACD'][index_value - 1] - df['MACD SIGNAL'][index_value - 1]):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### STOCHRSI
    if float(df['14 period stochastic RSI.'][index_value]) <= float(20):
        gauge_value += 1
    elif float(df['14 period stochastic RSI.'][index_value]) >= float(80):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### W%R
    if float(df['14 Williams %R'][index_value]) <= float(-80):
        gauge_value += 1
    elif float(df['14 Williams %R'][index_value]) >= float(-20):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### Bull/Bear Power
    if (df['Bull.'][index_value] + df['Bear.'][index_value]) > ((df['Bull.'][index_value - 1] + df['Bear.'][index_value - 1]) + .2):
        gauge_value += 1
    elif (df['Bull.'][index_value] + df['Bear.'][index_value]) < ((df['Bull.'][index_value - 1] + df['Bear.'][index_value - 1]) - .2):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### UO
    if float(df['UO'][index_value]) <= float(30):
        gauge_value += 1
    elif float(df['UO'][index_value]) >= float(70):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### For all periods of MAs except Ichimoku
    for i in 5, 10, 20, 30, 50, 100, 200:
        for j in 'SMA', 'EMA':
            if df['close'][index_value] > (df[str(i) + " period " + j][index_value] + .2):
                gauge_value += 1
            elif df['close'][index_value] < (df[str(i) + " period " + j][index_value] - .2):
                gauge_value -= 1
            else:
                gauge_value += 0
    if df['close'][index_value] > (df['20 period VAMA'][index_value] + .2):
        gauge_value += 1
    elif df['close'][index_value] < (df['20 period VAMA'][index_value] - .2):
        gauge_value -= 1
    else:
        gauge_value += 0
    if df['close'][index_value] > (df['9 period HMA.'][index_value] + .2):
        gauge_value += 1
    elif df['close'][index_value] < (df['9 period HMA.'][index_value] - .2):
        gauge_value -= 1
    else:
        gauge_value += 0
    ###### Ichimoku
    if df['close'][index_value] > (df['TENKAN'][index_value] > df['KIJUN'][index_value]):
        gauge_value += 1
    elif df['close'][index_value] < (df['TENKAN'][index_value] < df['KIJUN'][index_value]):
        gauge_value -= 1
    else:
        gauge_value += 0
    return gauge_value

In [6]:
def action(gauge_value):
    indicator_count=28
    gauge_value_averaged = (gauge_value / indicator_count)
    if (gauge_value_averaged >= -1.0) & (gauge_value_averaged < -0.6):
        print("Strong Sell, " + str(gauge_value_averaged))
    elif (gauge_value_averaged >= -0.6) & (gauge_value_averaged < -0.2):
        print("Sell, " + str(gauge_value_averaged))
    elif (gauge_value_averaged >= -0.2) & (gauge_value_averaged <= 0.2):
        print("Neutral, " + str(gauge_value_averaged))
    elif (gauge_value_averaged > 0.2) & (gauge_value_averaged <= 0.6):
        print("Buy, " + str(gauge_value_averaged))
    elif (gauge_value_averaged > 0.6) & (gauge_value_averaged <= 1.0):
        print("Strong Buy, " + str(gauge_value_averaged))
    

In [7]:
def stock_model(ticker):
    df = get_stock(ticker)
    df = create_features(df)
    action(get_gauge(df))

In [56]:
## Sentiment Values

# -1.0 to -0.6 = "Strong Sell"
# -0.6 to -0.2 = "Sell"
# -0.2 to 0.2 = "Neutral"
# 0.2 to 0.6 = "Buy"
# 0.6 to 1.0 = "Strong Buy"

In [10]:
stock_model('DELL')

Sell, -0.4642857142857143


In [9]:
def my_portfolio():
    ls = ['AMZN','AAPL','NVDA','TEAM','UNH','MDB','TWTR','MSFT','FB','TQQQ','COST','C','UBER','JACK','ABT','MGM']
    stonks = []
    for i in ls:
        stonks = stock_model(i)
    return stonks

In [10]:
my_portfolio()

Buy, 0.5357142857142857
Buy, 0.5
Strong Buy, 0.7142857142857143
Neutral, -0.14285714285714285
Neutral, 0.17857142857142858
Buy, 0.2857142857142857
Neutral, 0.10714285714285714
Strong Buy, 0.6785714285714286
Strong Buy, 0.6071428571428571
Strong Buy, 0.7142857142857143
Neutral, -0.17857142857142858
Neutral, 0.03571428571428571
Neutral, -0.07142857142857142
Neutral, 0.17857142857142858
Buy, 0.42857142857142855
Sell, -0.25


In [8]:
stock_model('NKLA')

Buy, 0.32142857142857145
